# Data 612 - Project 2 : Content-Based and Collaborative Filtering

#### Team Info 
+ Christina Valore, 
+ Juliann McEachern, 
+ Rajwant Mishra

#### Date: June 18, 2019

## Overview

For assignment 2, start with an existing dataset of user-item ratings, such as our toy books dataset, MovieLens, Jester [http://eigentaste.berkeley.edu/dataset/] or another dataset of your choosing.  Implement at least two of these recommendation algorithms: 
* Content-Based Filtering 
* User-User Collaborative Filtering 
* Item-Item Collaborative Filtering 
 

As an example of implementing a Content-Based recommender, you could build item profiles for a subset of MovieLens movies from scraping http://www.imdb.com/ or using the API at https://www.omdbapi.com/ (which has very recently instituted a small monthly fee).  A more challenging method would be to pull movie summaries or reviews and apply tf-idf and/or topic modeling. You should evaluate and compare different approaches, using different algorithms, normalization techniques, similarity methods, neighborhood sizes, etc.  You don’t need to be exhaustive—these are just some suggested possibilities.   You may use the course text’s [recommenderlab](https://cran.r-project.org/web/packages/recommenderlab/vignettes/recommenderlab.pdf) or any other library that you want.   Please provide at least one graph, and a textual summary of your findings and recommendations. 

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load data from local csv  into pandas dataframe


Bk_Rating = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/BX-Book-Ratings.csv')


Bk_Rating.head()

,"User-ID;""ISBN"";""Book-Rating"""
0,"276725;""034545104X"";""0"""
1,"276726;""0155061224"";""5"""
2,"276727;""0446520802"";""0"""
3,"276729;""052165615X"";""3"""
4,"276729;""0521795028"";""6"""


In [18]:
Bk_Book = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/BX-Books.csv',encoding="ISO-8859-1")

Bk_Book.head()

,"ISBN;""Book-Title"";""Book-Author"";""Year-Of-Publication"";""Publisher"";""Image-URL-S"";""Image-URL-M"";""Image-URL-L""",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,"0195153448;""Classical Mythology"";""Mark P. O. M...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"0002005018;""Clara Callan"";""Richard Bruce Wrigh...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"0060973129;""Decision in Normandy"";""Carlo D'Est...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"0374157065;""Flu: The Story of the Great Influe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"0393045218;""The Mummies of Urumchi"";""E. J. W. ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
Bk_User = pd.read_csv('https://raw.githubusercontent.com/jemceach/612-group/master/project-2/BX-Users.csv',encoding="ISO-8859-1")
Bk_User.head()

,A,B,C,D,E,F,G,H,I
0,"User-ID;""Location"";""Age""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"1;""nyc",new york,"usa"";NULL",NaN,NaN,NaN,NaN,NaN,NaN
2,"2;""stockton",california,"usa"";""18""",NaN,NaN,NaN,NaN,NaN,NaN
3,"3;""moscow",yukon territory,"russia"";NULL",NaN,NaN,NaN,NaN,NaN,NaN
4,"4;""porto",v.n.gaia,"portugal"";""17""",NaN,NaN,NaN,NaN,NaN,NaN
